In [1]:
!pip install minicons

  Using cached pandas-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
Using cached pandas-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Not uninstalling urllib3 at /nix/store/dcgjkj2z9zlmis91fq9py6c49rvasbnb-python3.12-urllib3-2.2.2/lib/python3.12/site-packages, outside environment /home/leo/git/miniconsPlayground/venv
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Not uninstalling pandas at /nix/store/6if4ab4fy227ww2ay90ddlnyp4d8dzgh-python3.12-pandas-2.2.2/lib/python3.12/site-packages, outside environment /home/leo/git/miniconsPlayground/venv
    Can't uninstall 'pandas'. No files were found to uninstall.

[notice] A 

In [2]:
import random
import os
from minicons import cwe

# Load stimuli and model

In [3]:
stimuli_folder = "./stimuli"
stimuli = []
for file in os.listdir(stimuli_folder):
    with open(stimuli_folder + "/" + file, 'r') as stimulus:
        file_contents = stimulus.read().replace('\n', '')
        text_split_at_space = file_contents.split(" ")
        while text_split_at_space.__contains__(""):
            text_split_at_space.remove("")
        stimuli.append(text_split_at_space)

model = cwe.CWE('bert-base-uncased')

/home/leo/git/miniconsPlayground/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Produce Sliding window context representations

In [12]:
def get_sliding_window_context_representations(data, window_size, layer):
    def get_context_words_for_pos(arr: list, pos: int) -> [str]:
        min_index = max(0, pos - (window_size + 1))
        words = arr[min_index]
        for i in range(min_index + 1, pos + 1):
            words = words + " " + arr[i]
        return [words, arr[pos]]

    context_and_representations = []
    for position in range(len(data)):
        context_words = get_context_words_for_pos(arr=data, pos=position)
        representation = model.extract_representation(context_words, layer=layer)
        context_and_representations.append([context_words, representation])
    return context_and_representations

## Extract context representations for one story

In [13]:
story = 0
context_size = 4
layer_to_extract = 12

story_representations = get_sliding_window_context_representations(data=stimuli[story],
                                                                   window_size=context_size,
                                                                   layer=layer_to_extract)

In [ ]:
random_index = random.randint(0, len(story_representations) - 1)

context = story_representations[random_index][0]
context_representation = story_representations[random_index][1]

print("Random context (index {0}) with its layer {1} representation:\n"
      "{2}\n"
      "{3}".format(random_index,
                   layer_to_extract,
                   context,
                   context_representation))

## Extract context representations for all stories

In [5]:
context_size = 5
layer_to_extract = None

all_representations = []

for stimulus in stimuli:
    representations = get_sliding_window_context_representations(data=stimulus, window_size=context_size,
                                                                 layer=layer_to_extract)
    all_representations.append(representations)

In [ ]:
print(all_representations)

# print(all_representations[0][0][1])